In [ ]:
#using the nlp
#import all the libaray
#also  
#1: unreliable,0: reliable
import numpy as np
import pandas as pd
import re #regular expression

import nltk
from nltk.corpus import stopwords  # remove all unneccesary prefix
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn .linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
nltk.download('stopwords')

In [145]:
a=stopwords.words('english')
print(a)
print(type(a))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [21]:
dataset=pd.read_csv('train.csv')
print(dataset['id'])
print(dataset['title'])
print(dataset.columns)

0            0
1            1
2            2
3            3
4            4
         ...  
20795    20795
20796    20796
20797    20797
20798    20798
20799    20799
Name: id, Length: 20800, dtype: int64
0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 20800, dtype: object
Index(['id', 'title', 'author', 'text', 'label'], dtype='object')


In [43]:
# dataset.shape[0]
# dataset.shape[1]
dataset.shape
dataset.size

104000

In [41]:
dataset.head()
dataset.tail()
dataset.sample(6)

,id,title,author,text,label
2267,2267,Taxpayers Will Defend Trump in Suit Charging C...,"Charlie Savage, Eric Lipton, Gardiner Harris a...",■ The federal taxpayer will be defending Presi...,0
5062,5062,Bob Creamer Admits To Daily Calls With The Cli...,Terresa Monroe-Hamilton,Bob Creamer Admits To Daily Calls With The C...,1
7232,7232,Chelsea Handler to Lead Anti-Trump ’Women’s Ma...,Daniel Nussbaum,Talk show host Chelsea Handler is set to lead ...,0
14250,14250,"John Glenn, American Hero of the Space Age, Di...",John Noble Wilford,"John Glenn, a son of Ohio who was hailed as ...",0
18419,18419,NaN,Jeannette,It’s always a pleasure 🙂,1
9943,9943,Within The Walls Of This Suburban Polling Plac...,NaN,Within The Walls Of This Suburban Polling Plac...,1


In [45]:
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [69]:
# # from the above either we drop the dataset
# import pandas as pd

# # Assuming your dataset is stored in a DataFrame named 'df'
# # Get the sum of null values in each column
# null_counts = dataset.isnull().sum()
# print(null_counts)

# # Get the names of columns with all null values

# empty_columns = null_counts[null_counts == dataset.shape[0]].index
# print(empty_columns)
# #dataset.shape[0].index
# # Drop the empty columns from the DataFrame
# df = dataset.drop(empty_columns, axis=1)
# print(df)
# df.shape

In [83]:
dataNew=dataset.fillna('')
print(dataNew)
dataNew['content']=dataNew['author']+ ' '+dataNew['title']
dataNew['content']
dataNew.columns

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

In [147]:
#X:Represents the independent variables
#y:Represents the dependent variable or target variable

X=dataNew.drop(columns='label',axis=1)
Y=dataNew['label']
print("the independent :",X)
print("the dependent :",Y)

the independent :           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                    

In [153]:
steamer=PorterStemmer()

In [ ]:
from nltk.stem import PorterStemmer

def stemming(content, stemmer):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stemmer.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

# Create a stemmer object
ps = PorterStemmer()

# Apply stemming to your data
dataNew['content'] = dataNew['content'].apply(stemming)
dataNew['content']

In [ ]:

X=data['content'].values
Y=data['label'].values
print(X)
print(Y)